In [145]:
from __future__ import print_function
import pandas as pd
import numpy as np
import dask
import os
from tqdm import tqdm, tqdm_pandas
tqdm_pandas(tqdm())

0it [00:00, ?it/s]


In [3]:
df_iter = pd.read_csv("data/result_df.csv",chunksize=100000,usecols=['path'],\
                      dtype = {"path" : "str"})

In [4]:
df = pd.concat(df_iter,ignore_index=True)

In [24]:
root_dir = "/mnt/processed/edgar_footnote_partition_indexing/word_index/"
path_ls = []
year_list = ['2010','2011','2012','2013']

for year in year_list : 
    year_dir = os.path.join(root_dir,year)
    month_list = sorted(os.listdir(year_dir))
    for month in month_list : 
        month_dir = os.path.join(year_dir,month)
        path_ls.append(os.path.join(month_dir,'item7.pkl'))

In [37]:
result_df = pd.DataFrame()

for idx,fname in enumerate(path_ls) : 
    chunk = pd.read_pickle(fname)[['HFSID','UPDATED_DAY','path']]
    result_df = pd.concat([result_df,chunk],ignore_index=True)

In [46]:
result_df['path'] = result_df.path.apply(lambda x : x.replace("locdisk",'mnt'))

In [54]:
merge_df = pd.merge(df,result_df,on='path')

In [56]:
merge_df.to_csv("data/merge_df.csv",index=False)

In [58]:
import sys
sys.path.append('../SUBMIT/')
from PKGS import us_starter 

In [60]:
univ1000,univ1000_id_list,price_return,sector,industry,sub_industry,date_list = \
us_starter.starter(20100101,20140101,'1000')

/usr/local/anaconda2/lib/python2.7/site-packages/numba/errors.py:102: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


In [63]:
return_df = pd.DataFrame(price_return,date_list,univ1000_id_list)

In [68]:
unique_df = merge_df.drop_duplicates().reset_index(drop=True)

In [110]:
def search(di,ii,date) : 
    di_start_idx = np.searchsorted(date_list,di,'right')-1
    di_start = date_list[di_start_idx]
    di_end = date_list[di_start_idx+date]
    
    try : 
        series = return_df.loc[:,ii]        
    except : 
        return np.nan
    val = (series.loc[di_start:di_end] + 1).cumprod().iloc[-1]
    return val

In [ ]:
unique_df['RETURN100'] = unique_df.apply(lambda x : search(x['UPDATED_DAY'],x['HFSID'],100),axis=1)

In [ ]:
unique_df.to_csv("data/return_df.csv",index=False)

In [131]:
label_df = pd.read_csv("data/groupby_df.csv")
label_df.rename(columns={'1':'SCORE'},inplace=True)

In [132]:
pd.merge(unique_df,label_df,on='path')[['RETURN5','RETURN10','RETURN20','SCORE']].dropna().corr()

,RETURN5,RETURN10,RETURN20,SCORE
RETURN5,1.000000,0.811281,0.657739,-0.042361
RETURN10,0.811281,1.000000,0.790621,-0.029953
RETURN20,0.657739,0.790621,1.000000,-0.020183
SCORE,-0.042361,-0.029953,-0.020183,1.000000


In [135]:
tmp_df = pd.read_csv("data/label_df.csv")
del tmp_df['0']

In [163]:
total_df = pd.merge(tmp_df,unique_df[['path','HFSID','UPDATED_DAY']],on='path')

In [164]:
total_df.to_csv('total_df.csv',index=False)

In [173]:
groupby_df = \
total_df.groupby('path').agg({'score':'mean','vader':'mean','HFSID':'median'})

In [174]:
groupby_df.to_csv('groupby_df.csv',index=False)